In [4]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [5]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True,
)

train_data.data = train_data.data.type(torch.float32)
train_data.targets = train_data.targets.type(torch.float32)

test_data.data = test_data.data.type(torch.float32)
test_data.targets = test_data.targets.type(torch.float32)

In [6]:
def fix_input(x):
    x =  x.flatten().reshape(1, 784)
    return x/255.0

# train_loader = DataLoader(dataset=train_data, batch_size=10, shuffle=False, num_workers=1)

# for idx, (data, target) in enumerate(train_loader):
#     print(data[0][0].shape)


class CustomDataSet:
    def __init__(self, data, targets, transform = None):
        self.data = data
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, target
    
customData = CustomDataSet(data=train_data.data, targets=train_data.targets, transform=fix_input)
train_loader = DataLoader(dataset=customData, batch_size=10, shuffle=True, num_workers=1)


class NeuralNet:

    def __init__(self, layers: list[int], lr):
        torch.manual_seed(1)
        self.biases = [torch.randn(1, layers[i], dtype=torch.float32, requires_grad=True) for i in range(1, len(layers))]
        self.weights = [torch.randn(layers[i], layers[i+1], dtype=torch.float32, requires_grad=True) for i in range(len(layers)-1)]
        self.lr = lr
        self.loss = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.weights+self.biases, lr = lr)

    def forward(self, x):
        for b,w in zip(self.biases, self.weights):
            x = torch.sigmoid(x@w+b)
        return x

    def conv(self, y):
        arr = [0]*10
        arr[int(y.item())] = 1.0
        return torch.tensor([arr])

    def predict(self, y):
        m = 0
        I = 0
        for i in range(10):
            if y[0][i] > m:
                I = i
                m = y[0][i]
        return I

    def train(self, training_loader, testing_data):
        for epoch in range(20):
            for idx, (data, targets) in enumerate(training_loader):
                for i in range(len(data)):
                    self.optimizer.zero_grad()
                    targets = targets.type(torch.float32)
                    y_pred = self.forward(data[i])
                    y = self.conv(targets[i])
                    l = self.loss(y_pred, y)
                    l.backward()
                    self.optimizer.step()
            correct = 0
            for i in range(len(testing_data.data)):
                y = self.forward(fix_input(testing_data.data[i]))
                z = self.predict(y)
                if z == testing_data.targets[i]:
                    correct += 1
            
            print(f"epoch {epoch}: {correct}/{len(testing_data.data)}")
        


n = NeuralNet([784, 30, 10], 3)


n.train(train_loader, test_data)

epoch 0: 9218/10000
epoch 1: 9272/10000
epoch 2: 9278/10000
epoch 3: 9425/10000
epoch 4: 9427/10000
epoch 5: 9439/10000
epoch 6: 9450/10000
epoch 7: 9485/10000
epoch 8: 9479/10000


KeyboardInterrupt: 